In [37]:
import os
import re
import random
import glob
import importlib
import sys
from bs4 import BeautifulSoup
import urllib
import urllib.request 
from urllib.request import urlopen
from urllib.request import Request
#from urllib2 import Request


import requests as req
from collections import defaultdict
import time

import pandas as pd
import numpy as np

import base64
import io

# visualization imports
from IPython.display import display
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline
sns.set() 

import ipywidgets as widgets
from ipywidgets import interact, interact_manual

from pdfminer.pdfparser import PDFParser
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfpage import PDFTextExtractionNotAllowed
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.layout import LAParams 
from pdfminer.converter import TextConverter


from sklearn import metrics
from sklearn.decomposition import PCA, LatentDirichletAllocation
from sklearn.preprocessing import scale, StandardScaler
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer,TfidfTransformer


import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
from nltk.tokenize import sent_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.probability import ConditionalFreqDist 
from nltk.probability import FreqDist 
nltk.download('stopwords')
nltk.download('punkt')
set(stopwords.words('spanish'))
nltk.download('wordnet')

from github import Github
from github import InputGitTreeElement


#We are using methods from this file 
%run ./AIVM_helper_classes.ipynb


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/aideenf/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/aideenf/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/aideenf/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [38]:
BASE_DIR_PDF_DOCS =  './Data/Iterative-models-building/Gathered_data/Conventions/pdf_docs/'

if not os.path.exists(BASE_DIR_PDF_DOCS):
    os.makedirs(BASE_DIR_PDF_DOCS)

    
NUM_KEYWORDS_PER_CONV = 30

In [45]:
# read data files for source data directly from github.
# to obtain the id for the folder, navigate the tree using
# https://api.github.com/repos/{owner}/{repo}/git/trees/master
# e.g https://api.github.com/repos/aideenf/AIVC/git/trees/master
# once navigated each directory will be of format
# https://api.github.com/repos/aideenf/AIVC/git/trees/{dir_sha}
#eg. https://api.github.com/repos/aideenf/AIVC/git/trees/392e7c49c4fc5825a54ab8c369997cfcd99b5fd0

gathered_pdf_files_list = []
gathered_pdf_files_dict = {}
pdf_list = []

# Defining up here so we can append several pd'fs
texts = []
data_provenance = []
convention = []
confidence_value = []
confidence_level = []
old = []
new = []

GATHERED_PDF_DIR = "https://api.github.com/repos/aideenf/AIVC/git/trees/08c1ae08e89a0add39df28517a66e45887d3501e"
resp = req.get(GATHERED_PDF_DIR)
response  = json.loads(resp.text)
for value in response['tree']:
    gathered_pdf_files_list.append(value['path'])
gathered_pdf_files_list = remove_excluded_files_except(gathered_pdf_files_list, ".pdf")
   
    
    
#Read pdf  file from github. 
for f in gathered_pdf_files_list:
    urlBase  ='https://github.com/aideenf/AIVC/raw/master/cp_wssc/Data/Iterative-models-building/Gathered_data/Conventions/pdf_docs/'
    gathered_pdf_files_dict[f] = urlBase + f
        
    

In [46]:
print (gathered_pdf_files_list)

['Ehealth2040.pdf', 'GE_Guidebook.pdf']


In [44]:
my_style = {'description_width': 'initial'}
my_layout = {'width': '500px'}
my_layout_short = {'width': '300px'}
pdf_to_parse = ""
convention = ""
convention_drop_down = widgets.Dropdown(
    options=['unknown','civic', 'domestic', 'green', 'industrial', 'inspired', 'market', 'project', 'renown'],
    value='unknown',
    description='Convention:',
    disabled=False,
    style = my_style,
    layout = my_layout
)

pdf_drop_down = widgets.Dropdown(
    options =  gathered_pdf_files_dict,
    description='File to parse:',
     #value=None,
    disabled=False,
    style = my_style,
    layout = my_layout
)

git_user = widgets.Text(
            description='GIT username',
            disabled=False,
            layout=my_layout_short,
            tooltip='enter the new value name',
            style = my_style
        )


git_pswd = widgets.Password(
            description='GIT password',
            disabled=False,
            layout=my_layout_short,
            tooltip='enter the new value name',
            style = my_style
        )


def output_function(convention_drop_down_value, pdf_drop_down_value, git_user_value, git_pswd_value ):
    print (convention_drop_down_value)
    print (pdf_drop_down_value)
    print (git_user_value)
    

output = widgets.interactive_output(output_function, {
            'convention_drop_down_value': convention_drop_down,
            'pdf_drop_down_value': pdf_drop_down,
            'git_user_value': git_user,
            'git_pswd_value': git_pswd
            })

display (convention_drop_down, pdf_drop_down, git_user, git_pswd, output)

Dropdown(description='Convention:', layout=Layout(width='500px'), options=('unknown', 'civic', 'domestic', 'gr…

Dropdown(description='File to parse:', layout=Layout(width='500px'), options={'Ehealth2040.pdf': 'https://gith…

Text(value='', description='GIT username', layout=Layout(width='300px'), style=DescriptionStyle(description_wi…

Password(description='GIT password', layout=Layout(width='300px'), style=DescriptionStyle(description_width='i…

Output()

# Read the pdf

In [6]:
url = pdf_drop_down.value
print (url)
#response = urllib.request.urlopen(Request(url)).read()
response = urllib.request.urlopen(url).read()
print (type (response))
memoryFile = io.BytesIO(response)
parser = PDFParser(memoryFile)
document = PDFDocument(parser)

# Try to parse the document
if not document.is_extractable:
    raise PDFTextExtractionNotAllowed

# Create a PDF resource manager object 
# that stores shared resources.
rsrcmgr = PDFResourceManager()

# Create a buffer for the parsed text
retstr = io.StringIO()

# Spacing parameters for parsing
laparams = LAParams()
codec = 'utf-8'
 
# Create a PDF device object
device = TextConverter(rsrcmgr, retstr, 
                        codec = codec, 
                        laparams = laparams)
# Create a PDF interpreter object
interpreter = PDFPageInterpreter(rsrcmgr, device)

# Process each page contained in the document.
for page in PDFPage.create_pages(document):
    interpreter.process_page(page)

print(type(document))
#tokenised_into_sentences = sent_tokenize(document)
#print(tokenised_into_sentences)
         
records = []

myStr = ""       
lines = retstr.getvalue().splitlines()
for line in lines:
    records.append(line)
    myStr = myStr + line

    


https://github.com/aideenf/AIVC/raw/master/cp_wssc/Data/Iterative-models-building/Gathered_data/Conventions/pdf_docs/GE_Guidebook.pdf
<class 'bytes'>
<class 'pdfminer.pdfdocument.PDFDocument'>


In [16]:


#tokenised_into_sentences = sent_tokenize(myStr)
tokenised_into_sentences = pre_process(myStr)
print ("number of sentences from pdf: ", len(tokenised_into_sentences))


for s in tokenised_into_sentences:
    if len(s) > 25 and len(s) < 500 and not "....." in s and not "download here citation" in s:
       
        s = pre_process_sentence(s)
        #print("******NEXT*******")
        #print(s)
        texts.append(str(s))
        data_provenance.append ("pdf")
        convention.append (convention_drop_down.value)
        confidence_value.append (0)
        confidence_level.append ("Not_from_Model")
        old.append (1)
        new.append (1)
        

        
print ("Final number of sentences after processing: ", len(texts))


#"text", "convention", "confidence_value", "confidence_level", "data_provenance"
pdf_text_df = pd.DataFrame(list(zip(texts, convention, data_provenance, confidence_value ,confidence_level)),
                columns = ("text", "convention", "data_provenance", "confidence_value", "confidence_level")) 
display(pdf_text_df.head(5))


#pdf_text_df_audited = pd.DataFrame(list(zip(texts, data_provenance, convention,  old , new)),
#               columns = ["text", "provenance", "convention", "old", "new"]) 


    

number of sentences from pdf:  897
Final number of sentences after processing:  694


,text,convention,data_provenance,confidence_value,confidence_level
0,a guidebook to the green economy issue green e...,green,pdf,0,Not_from_Model
1,these have been intensified by recent prolonge...,green,pdf,0,Not_from_Model
2,with governments today seeking effective ways ...,green,pdf,0,Not_from_Model
3,the concept has received significant internati...,green,pdf,0,Not_from_Model
4,this has resulted in a rapidly expanding liter...,green,pdf,0,Not_from_Model


In [17]:
## Now we will save a file 'audit_training_data.tsv'  and zipped version locally which contains all of the results
file_name = 'audit_data_from_pdf.tsv'
file_name_zip = 'audit_data_from_pdf.gz'

file_path = BASE_DIR_PDF_DOCS
pdf_text_df.to_csv(file_path + file_name, sep = '\t', index = False)
pdf_text_df.to_csv(file_path + file_name_zip, sep='\t', compression='gzip')

f = open( os.path.join(file_path, 'audit_data_from_pdf.csv'), 'rb') 


## We will also push this to git hub for use by audit tool
#Call helper file to zip the file for push to github
my_file_list = [file_path + file_name]
push_to_git_as = ['cp_wssc/Data/Iterative-models-building/Gathered_data/Conventions/pdf_docs/'+file_name]


GIT_USER = git_user.value 
GIT_PSWD = git_pswd.value 
GIT_REPO = 'AIVC'

print(GIT_USER, GIT_PSWD, GIT_REPO, my_file_list, push_to_git_as)

save_to_github(GIT_USER, GIT_PSWD, GIT_REPO, my_file_list, push_to_git_as)


aideenf aid99rk4 AIVC ['./Data/Iterative-models-building/Gathered_data/Conventions/pdf_docs/audit_data_from_pdf.tsv'] ['cp_wssc/Data/Iterative-models-building/Gathered_data/Conventions/pdf_docs/audit_data_from_pdf.tsv']
file to commit: ./Data/Iterative-models-building/Gathered_data/Conventions/pdf_docs/audit_data_from_pdf.tsv
push to git as: cp_wssc/Data/Iterative-models-building/Gathered_data/Conventions/pdf_docs/audit_data_from_pdf.tsv
got here 2
got here 3
File commited to github : GitCommit(sha="87c4bc0ec3a2ce36c8209033e7c6a34a4b9ccb4d")


In [20]:
document_lengths = np.array(list(map(len, pdf_text_df.text.str.split(' '))))

print("The average number of words in a sentence is: {}.".format(np.mean(document_lengths)))
print("The minimum number of words in a sentence is: {}.".format(min(document_lengths)))
print("The maximum number of words in a sentence is: {}.".format(max(document_lengths)))

The average number of words in a sentence is: 29.402017291066283.
The minimum number of words in a sentence is: 3.
The maximum number of words in a sentence is: 74.


In [28]:
!pip install PyPDF2
import PyPDF2

PDF = PyPDF2.PdfFileReader(memoryFile)

pages = PDF.getNumPages()
key = '/Annots'
uri = '/URI'
ank = '/A'

pdf_list = []

for page in range(pages):
    pageSliced = PDF.getPage(page)
    pageObject = pageSliced.getObject()
    if key in pageObject.keys():
        ann = pageObject[key]
        for a in ann:
            u = a.getObject()
            if uri in u[ank].keys():
                pdf_list.append(u[ank][uri])

pdf_list = keep_pdf_urls_only(pdf_list)
for pdf_link in pdf_list:
    print(pdf_link)

You should consider upgrading via the 'pip install --upgrade pip' command.
http://www.unep.ch/etb/pdf/2009%20statement%20deliver%20as%20one/Interagency%20Joint%20Statement.%20E%20rev1.pdf
http://www.unep.org/greeneconomy/Portals/88/documents/ger/ger_final_dec_2011/Green%20EconomyReport_Final_Dec2011.pdf
http://www.unemg.org/Portals/27/Documents/IMG/GreenEconomy/report/GreenEconomy-Full.pdf
http://www.unep.org/greeneconomy/Portals/88/documents/research_products/MeasuringProgress.pdf
http://www.unep.org/labour_environment/PDFs/Greenjobs/UNEPGreenJobs-E-Bookp1-30-Executivesummary.pdf
http://www.unep.org/pdf/green_economy_blue.pdf
http://www.unep.org/pdf/PressReleases/UNEP-ForestsGreenEco-basse_def_version_normale.pdf
http://www.unep.org/greeneconomy/Portals/88/documents/GEI%20Highlights/GE%20Trade%20Opportunities_executive-summary.pdf
http://pdf.wri.org/adapting_for_a_green_economy.pdf
http://www.unep.org/greeneconomy/Portals/88/documents/research_products/UN-DESA,%20UNCTAD%20Transition%2

In [ ]:
http://www.uwyo.edu/barbier/publications/a global green new deal-executive summery.pdf

# USE TF_IDF to create keywords per PDF
TF-IDF stands for “Term Frequency — Inverse Data Frequency”. 

Term Frequency (tf): gives us the frequency of the word in each document in the corpus. It is the ratio of number of times the word appears in a document compared to the total number of words in that document. It increases as the number of occurrences of that word within the document increases. Each document has its own tf.

Inverse Data Frequency (idf): used to calculate the weight of rare words across all documents in the corpus. The words that occur rarely in the corpus have a high IDF score. 

For equations see:
https://www.freecodecamp.org/news/how-to-process-textual-data-using-tf-idf-in-python-cd2bbc0a94a3/

In [35]:
cv=CountVectorizer(max_df=0.85,max_features=10000)
word_count_vector=cv.fit_transform([myStr])
list(cv.vocabulary_.keys())[:10]

ValueError: max_df corresponds to < documents than min_df

In [36]:
tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer.fit(tokenised_into_sentences)

ValueError: Expected 2D array, got 1D array instead:
array=['a guidebook to the green economy issue 1 green economy, green growth, and low carbon development history, de nitions and a guide to recent publicationsdivision for sustainable development, undesa this document was prepared by cameron allen and stuart clouth, un division for sustainable development, august 20121.'
 '2.'
 '3. contents introduction.....................................................................................................................5 related concepts green economy, green growth and low carbon development.............6 green economy................................................................................................................7 3.a.'
 'publications.................................................................................................................................. 9 towards a green economy pathways to sustainable development and poverty eradication unep........ 9 working 

In [31]:
def sort_coo(coo_matrix):
    tuples = zip(coo_matrix.col, coo_matrix.data)
    return sorted(tuples, key=lambda x: (x[1], x[0]), reverse=True)

 
def extract_topn_from_vector(feature_names, sorted_items, topn=10):
    """get the feature names and tf-idf score of top n items"""
    
    #use only topn items from vector
    sorted_items = sorted_items[:topn]
 
    score_vals = []
    feature_vals = []
    
    # word index and corresponding tf-idf score
    for idx, score in sorted_items:
        
        #keep track of feature name and its corresponding score
        score_vals.append(round(score, 3))
        feature_vals.append(feature_names[idx])
 
    #create a tuples of feature,score
    #results = zip(feature_vals,score_vals)
    results= {}
    for idx in range(len(feature_vals)):
        results[feature_vals[idx]]=score_vals[idx]
    
    return results
 

In [ ]:
# you only needs to do this once, this is a mapping of index to 
feature_names=cv.get_feature_names()

keywords_for_df = []
convention_for_df = []
for i,f in enumerate(aggregated_files):
    # get the document that we want to extract keywords from
    doc = docs[i]
    #generate tf-idf for the given document
    tf_idf_vector=tfidf_transformer.transform(cv.transform([doc]))

    #sort the tf-idf vectors by descending order of scores
    sorted_items=sort_coo(tf_idf_vector.tocoo())

    #extract only the top n; n here is 30
    keywords=extract_topn_from_vector(feature_names,sorted_items,NUM_KEYWORDS_PER_CONV)

    # now add the result to dataframe
    for k in keywords:
        convention_for_df.append(f.replace('aggregated_', '').replace('.tsv','').lower())
        keywords_for_df.append(k)

zippedList =  list(zip(keywords_for_df, convention_for_df))
keywords_df = pd.DataFrame(zippedList, columns = ['keyword' , 'convention'])
display (keywords_df.head(20))

# Store keywords as CSV file 

In [ ]:
path = KEYWORDS_DIR+'keywords.csv'
display ("storing keywords.csv at path:", path)
keywords_df.to_csv(path,index = None, header=True)

In [ ]:
keywords=extract_topn_from_vector(feature_names,sorted_items,15)